
---

### Promemoria: questo 📘 `.NET Interactive` deve essere eseguito da VS Code con [questi prerequisiti](../PREREQS.md).

---

<!-- # Recipe II: 🍽️ First Dish
## 🧑‍🍳 Let's cook our 🧂🔥 first basic dish

This notebook has been designed as your "classroom kitchen" to get you started quickly with this Semantic Kernel — the easy way to add LLM AI to your app. It's in three parts that is best described with technospeak so you get everything just right:

1. **Setting up your OpenAI or Azure OpenAI Service key.** This lets you use this notebook like a playground of sorts. And you only have to enter your key once to get going — it stores it locally into a file called "settings.json" on your disk. 🙀 Be sure to not let that file show up publicly anywhere like on your personal GitHub repo — so please .gitignore it.

2. **Getting a 🔥 kernel instantiated.** With your OpenAI or Azure OpenAI key you can then create a kernel to send instructions to. We've made it easy for you to use either OpenAI or Azure OpenAI. When using OpenAI, it will default to your using the `text-davinci-003` model; when you use Azure OpenAI there's an extra endpoint setting to consider — and in addition you're asked explicitly for the model you would like to use.

3. **Run a semantic 🧂 function.** Okay! You're ready to give your LLM AI a natural language prompt expressed as natural language. We call this kind of interaction with the model "semantic" because it lives in the world of the underlying meaning of the text you give to the model.  -->

# Ricetta II: 🍽️ Primo piatto
## 🧑‍🍳 Cuciniamo il nostro 🛢️🔥 primo piatto base

<!-- This notebook has been designed as your "classroom kitchen" to get you started quickly with this Semantic Kernel — the easy way to add LLM AI to your app. It's in three parts that is best described with technospeak so you get everything just right: -->

Questo quaderno è stato concepito per mostrare il modo più semplice per aggiungere l'intelligenza artificiale LLM a una vostra applicazione. 
È suddiviso in tre parti:

1. **Impostazione della chiave OpenAI o Azure OpenAI Service.** In questo modo è possibile utilizzare il notebook come una sorta di parco giochi. È sufficiente inserire la chiave una sola volta per iniziare: viene memorizzata localmente in un file chiamato "`settings.json`" sul disco. Assicuratevi che questo file non venga visualizzato pubblicamente da nessuna parte, come ad esempio sul vostro repo personale di GitHub.

2. **Ottenere un kernel 🔥 istanziato.** Con la chiave OpenAI o Azure OpenAI è possibile creare un kernel a cui inviare le istruzioni.
Quando si usa OpenAI, viene utilizzato per default il modello `text-davinci-003`; quando si usa Azure OpenAI c'è un'ulteriore impostazione dell'endpoint da considerare e inoltre viene chiesto esplicitamente il modello che si desidera utilizzare.

3. **Eseguire una semantic 🛢️ function.** Ok! Siete pronti a fare una richiesta in linguaggio naturale all'intelligenza artificiale LLM.

## Step 1️: Impostare la chiave OpenAI o Azure OpenAI Service

### 1.1: Il sistema vi chiederà di digitare la vostra chiave API segreta.

The boolean variable below `useAzureOpenAI` should be set to `false` if you are using a regular OpenAI key, and `true` if you have an Azure OpenAI Service key. Whichever way you set the variable, a rectangular box will appear at the 👆 top 👆 of the notebook page — asking you to enter the credentials.\
These credentials are stored locally on your own disk and don't go anywhere else. Once you hit the ▶️ button to run the code, be sure to 👆 look up to the top of this notebook page to enter your creds.

La variabile booleana sottostante `useAzureOpenAI` deve essere impostata su `false` se si utilizza una normale chiave OpenAI e su `true` se si dispone di una chiave Azure OpenAI Service. Qualunque sia l'impostazione della variabile, nella 👆 parte superiore 👆 della pagina apparirà un riquadro rettangolare che chiederà di inserire le credenziali.\
Queste credenziali sono memorizzate **localmente** sul proprio disco e non vanno da nessuna parte.

<!-- You will only see the ▶️ (play) when you hover over the code block below, or you click on it to select it.  -->

In [3]:
#!import ../config/Settings.cs

bool useAzureOpenAI = true;

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

Settings: OK: Azure OpenAI endpoint configured [../config/settings.json]
Settings: OK: deployment name configured [../config/settings.json]
Settings: OK: API key configured [../config/settings.json]


<!-- If you can see a confirmation line above that goes `Settings: OK: AI model configured ...` then proceed to step 2. Note that this is done only once. Your credentials are good to go. -->

RICORDA: è sufficiente eseguire questa operazione **una volta**. Il resto dei notebook utilizzerà la chiave API memorizzata sul vostro computer in questo importante passaggio.

<!-- ### 1.2 In case you mistyped your API key information, follow these instructions -->
### 1.2 Nel caso in cui si sia commesso un errore di immissione, seguire le seguenti istruzioni

#### 🟡 **SALTA QUESTO STEP** se la chiave è già stata impostata con successo

Premere il pulsante ▶️ solo se si desidera ripristinare quanto inserito in precedenza. Prima di farlo, è necessario "decommentare" due caratteri eliminando `//` all'inizio della riga 4.

In altre parole, cambiare `// Settings.Reset();` in `Settings.Reset();`. In questo modo si "decommenta" quella riga di codice, in modo che diventi attiva.

In [6]:
#!import ../config/Settings.cs

// Uncomment the line below to reset your settings and run step 1.1 again so it asks you for your API key
// Settings.Reset();

Settings deleted. Run the notebook again to configure your AI backend.


## Step 2️: Preparare un kernel 🔥 per cucinare il primo piatto

Congratulazioni! Siete a un terzo del percorso! Premete ▶️ per accedere alle credenziali memorizzate localmente, impostate nel primo passo. Questo passo carica il pacchetto Microsoft.SemanticKernel e prepara il resto del blocco note per far funzionare subito la funzione semantica.

In [8]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. 
// Name the service as "davinci" — assuming that you're using one of the davinci completion models. 
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.9.61.1-preview

Se si vede una linea di conferma in alto che inizia con `Installed Packages ... • Microsoft.SemanticKernel, ...` procedere al punto 3.

<!-- 😱 **Get an error message?** The most common error is accidentally resetting your credentials file, or setting it up with the wrong information. Look at the 'settings.json' file in your config directory to make sure your settings make sense. And if you're still stuck, go to https://aka.ms/sk/discord where we have realtime support. -->

## Step 3️: Eseguire una funzione semantica 🛢️ in Semantic Kernel per 🔥cuocere 

<!-- Before you set off to write a semantic function, review our documentation on semantic functions at our [learning hub](https://learn.microsoft.com/en-us/semantic-kernel/howto/semanticfunctions). You'll want to feel comfortable with two ideas: -->

Bastano questi due concetti:

* Creare un prompt **parametrizzato** con una sola variabile - `$input` è la variabile di input predefinita. \[[Approfondimento](https://learn.microsoft.com/semantic-kernel/howto/semanticfunctions#writing-a-more-powerful-templated-prompt)\]

* Configurare il prompt con alcune **impostazioni standard** - `MaxTokens`, `Temperature`, `TopP`. \[[Approfondimento](https://learn.microsoft.com/en-us/semantic-kernel/howto/configuringfunctions)\]

Mancano solo tre passi all'esecuzione di una funzione semantica. Preparatevi!

### Step 3.1: Definire un prompt parametrizzato che accetta un singolo input

Il codice seguente è quello che chiamiamo "funzione semantica", che è quasi equivalente alla parola "prompt". 
Si sentirà parlare di questi termini in modo intercambiabile. Inoltre, sentirete la frase "skill semantica" e vi chiederete: "È la stessa cosa di una funzione semantica?". No, ma è sono collegato. Non preoccupatevi e continuate ad andare avanti!

In [9]:
string mySemanticFunctionInline = """
{{$input}}

Summarize the content above in less than 140 characters.
""";

Console.WriteLine("A string has been set to be used as a semantic function.");

A string has been set to be used as a semantic function.


Quindi la funzione semantica che stiamo per definire prende un testo `$input` e lo riassume in meno di 140 caratteri.

### Step 3.2: Configurare il prompt per renderlo più non deterministico (creativo) o deterministico (diretto)

#### 🔵 FAST TRACK Most people will just run the code below without much thought to tuning it, to start

L'impostazione `MaxTokens` determina il numero massimo di tokens da generare nel completamento. È il parametro più importante [setting](https://learn.microsoft.com/en-us/semantic-kernel/howto/configuringfunctions) da conoscere, perché ha un impatto su quanto spendete per ogni richiesta.

Inoltre, è possibile modellare in modo sottile l'uscita della risposta con gli altri due parametri. Per rendere la risposta più o meno "creativa", si può regolare l'impostazione `Temperatura' tra 0 e 1`. È inoltre possibile impostare il parametro `TopP` tra 0 (vocabolario più piccolo) e 1 (vocabolario più grande) per ottenere un risultato diverso.

In [10]:
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.SemanticFunctions;

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 1000, Temperature = 0.2, TopP = 0.5,
    }
};

var promptTemplate = new PromptTemplate(
    mySemanticFunctionInline, promptConfig, kernel
);

var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

// "MySkill" is the name of the skill
// "Summary" is the name of the semantic function
var summaryFunction = kernel.RegisterSemanticFunction("MySkill", "Summary", functionConfig);

Console.WriteLine("A semantic function has been registered.");


A semantic function has been registered.


### Step 3.3: Set your input to the templated prompt and have the kernel 🔥 process it

In [12]:

var input = """
I think with some confidence I can say that 2023 is going to be the most exciting year that 
the AI community has ever had,” writes Kevin Scott, chief technology officer at Microsoft, 
in a Q&A on the company’s AI blog. He acknowledges that he also thought 2022 was the most 
exciting year for AI, but he believes that the pace of innovation is only increasing. 
This is particularly true with generative AI, which doesn’t simply analyze large data sets 
but is a tool people can use to create entirely new works. We can already see its promise 
in systems like GPT-3, which can do anything from helping copyedit and summarize text to 
providing inspiration, and DALL-E 2, which can create useful and arresting works of art 
based on text inputs. Here are some of Scott’s predictions about how AI will change the 
way we work and play.
""";
// Text source: https://www.microsoft.com/en-us/worklab/kevin-scott-on-5-ways-generative-ai-will-transform-work-in-2023

var summary = await kernel.RunAsync(input, summaryFunction);

Console.WriteLine(summary);


2023 will be the most exciting year for AI yet, with generative AI tools like GPT-3 and DALL-E 2 creating new works from text inputs.


🎉 Hai appena creato la tua prima funzione semantica usando Semantic Kernel. Congratulazioni 🔥!

<!-- > ✅ Be sure to use `text-davinci-003` instead of the more trendy `gpt-3.5-turbo` when you run the above. -->

> 🤔 **Ottieni il seguente `"Error: Throttling: Too many requests ..."` messaggio?** I servizi OpenAI risultano essere estremamente popolari in questi giorni. Se si utilizza la chiave per un account gratuito, questo messaggio apparirà spesso.

<!-- > 😱 **Get a different error message?** If you can't see a summarization of the text above, then go to https://aka.ms/sk/discord where we have realtime support available to troubleshoot your problem. -->

# ⏭️ I prossimi passi

Esempi più avanzati nei notebook disponibili nel repo GitHub all'indirizzo [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Vediamo le 🛢️ skills!](../e3-skills-rack/notebook.ipynb)

<!-- Or stay a longer while and change the prompt above to your liking; and also the `$input` and other parameters to your liking. Please keep in mind that each API call to OpenAI or Azure OpenAI Services will use up tokens. -->